<a href="https://colab.research.google.com/github/0chandansharma/Abstractive-summarization/blob/master/h5topb_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [2]:
tf.__version__


'1.15.0'

In [3]:
pip install tensorflow==1.6.0


     |████████████████████████████████| 45.9MB 47kB/s 
     |████████████████████████████████| 3.1MB 37.0MB/s 
     |████████████████████████████████| 890kB 44.1MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp27-none-any.whl size=107220 sha256=aa0f30d6c7ccd514149395ab8a8a0a67aa6163d1b5cca863abbdf38759a5016d
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
ERROR: fastai 0.7.0 has requirement torch<0.4, but you'll have torch 1.3.0+cu100 which is incompatible.
ERROR: magenta 0.3.19 has requirement tensorflow>=1.12.0, but you'll have tensorflow 1.6.0 which is incompatible.
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.1.0
    Uninstalling bleach-3.1.0:
      Successfully uninstalled bleach-3.1.0
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboa

In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'1.6.0'

In [3]:
pip install keras_retinanet

     |████████████████████████████████| 61kB 2.0MB/s 
  Created wheel for keras-retinanet: filename=keras_retinanet-0.5.1-cp27-cp27mu-linux_x86_64.whl size=173714 sha256=8ea74a1cc0fba5ac5017d305dc0f9868a89cd2ffac3b1d916ea4eef7790e1025
  Stored in directory: /root/.cache/pip/wheels/d9/a5/98/87f8ed08913b184bb625dde1c1277a4dfe969c77b377abde69
  Created wheel for keras-resnet: filename=keras_resnet-0.2.0-py2.py3-none-any.whl size=20485 sha256=eb2da8f62cbca49f1ab8db522a3ccd6fc0abba6fa02ac23366d9176e8e31de1e
  Stored in directory: /root/.cache/pip/wheels/5f/09/a5/497a30fd9ad9964e98a1254d1e164bcd1b8a5eda36197ecb3c
Successfully built keras-retinanet keras-resnet


In [4]:
from keras import backend as K

Using TensorFlow backend.


In [0]:
from keras_retinanet.models import load_model

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [24]:
model = load_model('/content/drive/My Drive/output.h5')

/usr/local/lib/python2.7/dist-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [25]:
print(model.outputs)
# [<tf.Tensor 'dense_2/Softmax:0' shape=(?, 10) dtype=float32>]
print(model.inputs)

[<tf.Tensor 'filtered_detections_2/map/TensorArrayStack/TensorArrayGatherV3:0' shape=(?, 300, 4) dtype=float32>, <tf.Tensor 'filtered_detections_2/map/TensorArrayStack_1/TensorArrayGatherV3:0' shape=(?, 300) dtype=float32>, <tf.Tensor 'filtered_detections_2/map/TensorArrayStack_2/TensorArrayGatherV3:0' shape=(?, 300) dtype=int32>]
[<tf.Tensor 'input_1_2:0' shape=(?, ?, ?, 3) dtype=float32>]


In [0]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
  
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph


In [27]:
frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

INFO:tensorflow:Froze 918 variables.
Converted 918 variables to const ops.


In [29]:
# Save to ./model/tf_model.pb
tf.train.write_graph(frozen_graph, "/content/drive/My Drive/model", "output_model.pb", as_text=False)

'/content/drive/My Drive/model/output_model.pb'